## Machine learning models

The experiment was repeated 1000 times for 5-fold cross validation

Five machine learning algorithms are used: SVM, LR, RF, XGBoost, AdaBoost

The performance evaluation indexes of the five models were calculated: acc, F1-score, precision, recall, auc, ap

ROC curve and PR curve of the model were drawn for 1000 times

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

In [1]:
# Fifty fold cross validation, 1000 repeats of the experiment
def cv_repeat_model(cv, repeat, random_state, feature, y, model):
    
    cv_r = cv * repeat # cv-fold cross validation, repeat times
    rkf = RepeatedKFold(n_splits=cv, n_repeats=repeat, random_state=100) # Repeat the cv-fold data partition repeat times
    
    x_train_r = []
    x_test_r = []
    y_train_label_r = []
    y_test_label_r = []
    for train_index_r,test_index_r in rkf.split(feature):
        train_set_r = feature[train_index_r,:]
        test_set_r = feature[test_index_r,:]
        y_train = y[train_index_r]
        y_test = y[test_index_r]
        # Stack all five-fold results
        x_train_r.append(train_set_r)
        x_test_r.append(test_set_r)
        y_train_label_r.append(y_train)
        y_test_label_r.append(y_test)
    
    # training and predict
    y_proba_label_r = []
    for i in range(cv_r):
        model.fit(x_train_r[i], y_train_label_r[i])   # model corresponds to five models respectively
        y_proba_r = model.predict_proba(x_test_r[i])   
        y_proba_r = np.array(list(y_proba_r[:, 1]))  
        y_proba_label_r.append(y_proba_r)
     
    return cv_r, x_train_r, x_test_r, y_train_label_r, y_test_label_r, y_proba_label_r

In [2]:
# Figure out the probability and true classification of class 0
def change_y(y_true, y_pred):
    # Splice five folds
    yy = np.concatenate(y_true).reshape(repeat, -1)  # Stack the 5 fold models for each training session
    pp = np.concatenate(y_pred).reshape(repeat, -1)
    
    y_true_r = []
    y_pred_r = []
    for i in range(len(repeat)):
        y_true_k = pd.DataFrame(yy[i]).rename(columns={0:'y_1'})
        y_pred_k = pd.DataFrame(pp[i]).rename(columns={0:'pred_1'})
        y = pd.concat([y_true_k, y_pred_k], axis=1)

        y['pred_0'] = ""
        y['y_0'] = ""
        y_true_i, y_pred_i = y_0_1(y)
        y_true_r.append(y_true_i)
        y_pred_r.append(y_pred_i)
        
    return y_true_r, y_pred_r

In [3]:
# Compute micro-average ROC curve and ROC area
def plot_roc_curve(y_true_r, y_pred_r, text_list, path):
    # Calculate ROC curve parameters
    mean_fpr_micro = np.linspace(0, 1, 100)
    fprs_micro = []
    tprs_micro = []
    auc_micro = []
    for k in range(len(y_true_r)):
        # Calculate the parameters required for the curve
        fpr, tpr, thres = roc_curve(y_true_r[k].ravel(), y_pred_r[k].ravel()) 
       
        fprs_micro.append(fpr)
        
        interp_tpr = np.interp(mean_fpr_micro, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs_micro.append(interp_tpr)
        
    mean_tpr_micro = np.mean(tprs_micro, axis=0)
    mean_tpr_micro[-1] = 1.0
    std_tpr_micro = np.std(tprs_micro, axis=0)
    
    # Minimum and maximum values of 1000 repetitions, upper and lower limits of the interval of the ROC curve
    tprs_lower_micro = np.maximum(mean_tpr_micro - std_tpr_micro, 0)
    tprs_upper_micro = np.minimum(mean_tpr_micro + std_tpr_micro, 1)
    
    for k in range(len(y_true_r)):
        micro = roc_auc_score(y_true_r[k], y_pred_r[k], average='micro', multi_class='ovr')
        auc_micro.append(micro)
    
    auc_mean_micro = np.mean(auc_micro)
    auc_std_micro = np.std(auc_micro)
    
    # Draw ROC curve
    size = 20
    plt.figure()

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey', alpha=.8)
    
    plt.plot(mean_fpr_micro, mean_tpr_micro, color='#2878B5',label=r'AUC = %0.3f $\pm$ %0.3f' % (auc_mean_micro, auc_std_micro),lw=2, alpha=1)
    plt.fill_between(mean_fpr_micro, tprs_lower_micro, tprs_upper_micro, color='#82B0D2', alpha=.6, label=r'$\pm$ 1 std. dev.')

    ax = plt.gca()
    ax.set(xlim=[-0.02, 1.02], ylim=[-0.02, 1.02])
    plt.legend(loc="lower right",fontsize=size-2)
    plt.title(text_list,weight='bold', fontdict={'size':size})
    plt.ylabel('True positive rate',weight='bold', fontdict={'size':size-2})
    plt.xlabel('False positive rate',weight='bold', fontdict={'size':size-2})
    
    plt.savefig(path, transparent=False, dpi=600)

In [4]:
def plot_pr_curve(y_true_r, y_pred_r, text_list, save_path):
    # Calculate PR curve parameters
    size=18
    mean_rc = np.linspace(0, 1, 100)
    prs = []
    aucs = []
    for k in range(len(y_true_r)):
        # Calculate the parameters required for the curve
        pr, rc, thres = precision_recall_curve(y_true_r[k][:,1], y_pred_r[k][:,1], pos_label=1)
        pr, rc = pr[::-1], rc[::-1]
        interp_pr = np.interp(mean_rc, rc, pr)
        interp_pr[0] = 1.0 
        prs.append(interp_pr)
        
        AUC = average_precision_score(y_true_r[k][:,1], y_pred_r[k][:,1], pos_label=1)
        aucs.append(AUC)
        
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)

    mean_pr = np.mean(prs, axis=0)
    std_pr = np.std(prs, axis=0)
    
    # Minimum and maximum values of 1000 repetitions, upper and lower limits of the interval of the PR curve
    prs_upper = np.minimum(mean_pr + std_pr, 1)
    prs_lower = np.maximum(mean_pr - std_pr, 0)

    
    # Draw PR curve
    plt.figure()
    
    plt.plot(mean_rc, mean_pr, color='#FFA500',label='%s (AP=%0.3f $\pm$ %0.3f)' % (model, mean_auc, std_auc),lw=2, alpha=1)
    plt.fill_between(mean_rc, prs_lower, prs_upper, color='#FFD700', alpha=.6, label=r'$\pm$ 1 std. dev.')
    plt.plot([0, 1], [np.min(mean_pr), np.min(mean_pr)], linestyle='--', label=r'Baseline (AP=%0.3f)' % (np.min(mean_pr)), lw=2, color='grey', alpha=.8)

    ax = plt.gca()
    ax.set(xlim=[-0.02, 1.02], ylim=[-0.02, 1.02])
    plt.legend(loc="lower right",fontsize=size-2)
    plt.title(text_list,weight='bold', fontdict={'size':size})
    plt.ylabel('Precision',weight='bold', fontdict={'size':size-2})
    plt.xlabel('Recall',weight='bold', fontdict={'size':size-2})
    
    plt.savefig(save_path, transparent=False, dpi=600)

In [5]:
# Calculate the model performance evaluation index
def report(y_true, y_pred):
    precision = []
    recall = []
    f1 = []
    acc = []
    auc = []
    ap = []
    for i in range(len(y_true)):
        result = pd.DataFrame(classification_report(y_true[i][:,1], y_pred[i][:,1]>0.5, output_dict=True))
        precision.append(result['weighted avg']['precision'])
        recall.append(result['weighted avg']['recall'])
        f1.append(result['weighted avg']['f1-score'])
        acc.append(result['accuracy']['support'])
        auc.append(roc_auc_score(y_true[i], y_pred[i], average='micro', multi_class='ovr'))
        ap.append(average_precision_score(y_true[i][:,1], y_pred[i][:,1], pos_label=1))
    
    # The mean value, standard deviation and 95% confidence interval of seven evaluation indexes were calculated respectively
    print('auc:{0}, {1}, {2}'.format(np.mean(auc), np.std(auc), np.percentile(auc, (2.5, 97.5))))
    print('ap:{0}, {1}, {2}'.format(np.mean(ap), np.std(ap), np.percentile(ap, (2.5, 97.5))))
    print('recall:{0}, {1}, {2}'.format(np.mean(recall), np.std(recall), np.percentile(recall, (2.5, 97.5))))
    print('precision:{0}, {1}, {2}'.format(np.mean(precision), np.std(precision), np.percentile(precision, (2.5, 97.5))))
    print('f1:{0}, {1}, {2}'.format(np.mean(f1), np.std(f1), np.percentile(f1, (2.5, 97.5))))
    print('acc:{0}, {1}, {2}'.format(np.mean(acc), np.std(acc), np.percentile(acc, (2.5, 97.5))))

## The two-stage classification tasks are as follows:

#### HC vs DOC

In [8]:
# X
feature_table = pd.read_csv('./data/fc_feature.csv')
feature = np.array(feature_table)

In [9]:
# y
y = np.array(pd.read_csv('./data/y_label.csv')['y'])

1. SVM

In [ ]:
svc = SVC(kernel='linear', random_state=0, probability=True)
cv_svc, x_train_svc, x_test_svc, y_train_label_svc, y_test_label_svc, y_proba_label_svc = cv_repeat_model(5, 1000, 100, feature, y, svc)
y_true_svc, y_pred_svc = change_y(y_test_label_svc, y_proba_label_svc)
plot_roc_curve(y_test_label_svc, y_proba_label_svc, 'SVM', './SVM_doc_control.tif')
plot_pr_curve(y_test_label_svc, y_proba_label_svc, 'SVM', './SVM_doc_control.tif')
report(y_test_label_svc, y_proba_label_svc)

2. LR

In [ ]:
lr = LogisticRegression(random_state=0)
cv_lr, x_train_lr, x_test_lr, y_train_label_lr, y_test_label_lr, y_proba_label_lr = cv_repeat_model(5, 1000, 100, feature, y, lr)
y_true_lr, y_pred_lr = change_y(y_test_label_lr, y_proba_label_lr)
plot_roc_curve(y_test_label_lr, y_proba_label_lr, 'LR', './LR_doc_control.tif')
plot_pr_curve(y_test_label_svc, y_proba_label_svc, 'LR', './LR_doc_control.tif')
report(y_test_label_lr, y_proba_label_lr)

3. RF

In [ ]:
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
cv_rf, x_train_rf, x_test_rf, y_train_label_rf, y_test_label_rf, y_proba_label_rf = cv_repeat_model(5, 1000, 100, feature, y, rf)
y_true_rf, y_pred_rf = change_y(y_test_label_rf, y_proba_label_rf)
plot_roc_curve(y_test_label_rf, y_proba_label_rf, 'RF', './RF_doc_control.tif')
plot_pr_curve(y_test_label_rf, y_proba_label_rf, 'RF', './RF_doc_control.tif')
report(y_test_label_rf, y_proba_label_rf)

4. XGBoost

In [ ]:
xg = XGBClassifier(n_estimators=1000, random_state=0, eval_metric='mlogloss', use_label_encoder=False)
cv_xg, x_train_xg, x_test_xg, y_train_label_xg, y_test_label_xg, y_proba_label_xg = cv_repeat_model(5, 1000, 100, feature, y, xg)
y_true_xg, y_pred_xg = change_y(y_test_label_xg, y_proba_label_xg)
plot_roc_curve(y_test_label_xg, y_proba_label_xg, 'XGBoost', './XGBoost_doc_control.tif')
plot_pr_curve(y_test_label_xg, y_proba_label_xg, 'XGBoost', './XGBoost_doc_control.tif')
report(y_test_label_xg, y_proba_label_xg)

5. AdaBoost

In [ ]:
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
cv_ada, x_train_ada, x_test_ada, y_train_label_ada, y_test_label_ada, y_proba_label_ada = cv_repeat_model(5, 1000, 100, feature, y, ada)
y_true_ada, y_pred_ada = change_y(y_test_label_ada, y_proba_label_ada)
plot_roc_curve(y_test_label_ada, y_proba_label_ada, 'AdaBoost', './AdaBoost_doc_control.tif')
plot_pr_curve(y_test_label_ada, y_proba_label_ada, 'AdaBoost', './AdaBoost_doc_control.tif')
report(y_test_label_ada, y_proba_label_ada)

#### MCS vs UWS

In [ ]:
feature_2 = feature[:51, :]
y_2 = y[:51]

1. SVM

In [ ]:
svc_2 = SVC(kernel='linear', random_state=0, probability=True)
cv_svc_2, x_train_svc_2, x_test_svc_2, y_train_label_svc_2, y_test_label_svc_2, y_proba_label_svc_2 = cv_repeat_model(5, 1000, 100, feature_2, y_2, svc_2)
y_true_svc_2, y_pred_svc_2 = change_y(y_test_label_svc_2, y_proba_label_svc_2)
plot_roc_curve(y_test_label_svc_2, y_proba_label_svc_2, 'SVM', './SVM_mcs_vs.tif')
plot_pr_curve(y_test_label_svc_2, y_proba_label_svc_2, 'SVM', './SVM_mcs_vs.tif')
report(y_test_label_svc_2, y_proba_label_svc_2)

2. LR

In [ ]:
lr_2 = LogisticRegression(random_state=0)
cv_lr_2, x_train_lr_2, x_test_lr_2, y_train_label_lr_2, y_test_label_lr_2, y_proba_label_lr_2 = cv_repeat_model(5, 1000, 100, feature_2, y_2, lr_2)
y_true_lr_2, y_pred_lr_2 = change_y(y_test_label_lr_2, y_proba_label_lr_2)
plot_roc_curve(y_test_label_lr_2, y_proba_label_lr_2, 'LR', './LR_mcs_vs.tif')
plot_pr_curve(y_test_label_svc_2, y_proba_label_svc_2, 'LR', './LR_mcs_vs.tif')
report(y_test_label_lr_2, y_proba_label_lr_2)

3. RF

In [ ]:
rf_2 = RandomForestClassifier(n_estimators=1000, random_state=0)
cv_rf_2, x_train_rf_2, x_test_rf_2, y_train_label_rf_2, y_test_label_rf_2, y_proba_label_rf_2 = cv_repeat_model(5, 1000, 100, feature_2, y_2, rf_2)
y_true_rf_2, y_pred_rf_2 = change_y(y_test_label_rf_2, y_proba_label_rf_2)
plot_roc_curve(y_test_label_rf_2, y_proba_label_rf_2, 'RF', './RF_mcs_vs.tif')
plot_pr_curve(y_test_label_rf_2, y_proba_label_rf_2, 'RF', './RF_mcs_vs.tif')
report(y_test_label_rf_2, y_proba_label_rf_2)

4. XGBoost

In [ ]:
xg_2 = XGBClassifier(n_estimators=1000, random_state=0, eval_metric='mlogloss', use_label_encoder=False)
cv_xg_2, x_train_xg_2, x_test_xg_2, y_train_label_xg_2, y_test_label_xg_2, y_proba_label_xg_2 = cv_repeat_model(5, 1000, 100, feature_2, y_2, xg_2)
y_true_xg_2, y_pred_xg_2 = change_y(y_test_label_xg_2, y_proba_label_xg_2)
plot_roc_curve(y_test_label_xg_2, y_proba_label_xg_2, 'XGBoost', './XGBoost_mcs_vs.tif')
plot_pr_curve(y_test_label_xg_2, y_proba_label_xg_2, 'XGBoost', './XGBoost_mcs_vs.tif')
report(y_test_label_xg_2, y_proba_label_xg_2)

5. AdaBoost

In [ ]:
ada_2 = AdaBoostClassifier(n_estimators=1000, random_state=0)
cv_ada_2, x_train_ada_2, x_test_ada_2, y_train_label_ada_2, y_test_label_ada_2, y_proba_label_ada_2 = cv_repeat_model(5, 1000, 100, feature_2, y_2, ada_2)
y_true_ada_2, y_pred_ada_2 = change_y(y_test_label_ada_2, y_proba_label_ada_2)
plot_roc_curve(y_test_label_ada_2, y_proba_label_ada_2, 'AdaBoost', './AdaBoost_mcs_vs.tif')
plot_pr_curve(y_test_label_ada_2, y_proba_label_ada_2, 'AdaBoost', './AdaBoost_mcs_vs.tif')
report(y_test_label_ada_2, y_proba_label_ada_2)